In [16]:
!pip install pytesseract io pdf2image opencv-python

In [ ]:
!pip install scikit-image


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import fitz  # PyMuPDF
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim
import io 



# Path to the video file
video_path = '/content/drive/MyDrive/Recording #3.mp4'

# Initialize the video capture object
cap = cv2.VideoCapture(video_path)

# Initialize a list to hold the unique slides
unique_slides = []

# Previous frame for comparison
prev_frame = None

# Tolerance to decide whether two images are similar
similarity_threshold = 0.95

def are_frames_similar(frame1, frame2, threshold):
    # Calculate the structural similarity index (SSI) between two frames
    # You can use skimage's structural_similarity function for this
    # from skimage.metrics import structural_similarity as compare_ssim
    # Return True if they are similar above the given threshold
        # Compute SSIM between two images
    ssi_index, _ = ssim(frame1, frame2, full=True)
    return ssi_index > threshold

    pass

# Read frames from the video
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale for comparison
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compare the current frame with the previous frame
    if prev_frame is not None:
        if not are_frames_similar(prev_frame, gray_frame, similarity_threshold):
            # Frames are not similar; save the current frame as a unique slide
            slide_image = Image.fromarray(frame)
            unique_slides.append(slide_image)

    prev_frame = gray_frame
    frame_count += 1

# Release the video capture object
cap.release()



NameError: ignored

In [ ]:
import io
# Create a PDF document
pdf_path = 'extracted_slides.pdf'
pdf_doc = fitz.open()

# Add slides to the PDF
for slide in unique_slides:
    img_byte_arr = io.BytesIO()
    slide.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    img = fitz.open("png", img_byte_arr)
    rect = img[0].rect
    pdfbytes = img.convert_to_pdf()
    img.close()
    imgPDF = fitz.open("pdf", pdfbytes)
    page = pdf_doc.new_page(width = rect.width, height = rect.height)
    page.show_pdf_page(rect, imgPDF, 0)

# Save the PDF document
pdf_doc.save(pdf_path)
pdf_doc.close()

print(f"PDF with extracted slides created at: {pdf_path}")


PDF with extracted slides created at: extracted_slides.pdf
